# **Fine-Tuning the Turnsense Model (latishab/turnsense)**
In this notebook we load the published turnsense model (which is based on SmolLM2-135M and already fine‑tuned for turn detection) from Hugging Face, and then further fine‑tune it on an additional dataset.

**Steps:**
1.   Load the tokenizer and dataset.
2.   Preprocess and format the dataset.
3. Load the turnsense model.
4. (Optional) Re-apply or check the adapter configuration.
5. Fine-tune using SFTTrainer.
6. Save the newly fine‑tuned adapter/model.


In [1]:
!pip install transformers datasets peft trl wandb bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd

## 1. Load Tokenizer and Dataset
tokenizer = AutoTokenizer.from_pretrained("latishab/turnsense")
dataset = load_dataset("latishab/turns-2k")["train"]

# Convert to pandas DataFrame for stratified splitting
df = dataset.to_pandas()

# Stratified split (55% train, 45% test)
train_df, test_df = train_test_split(
    df,
    test_size=0.20,
    stratify=df["label"],
    random_state=42
)

# Convert back to Hugging Face Dataset format
ds = {
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
}
print(f"Dataset split: {len(ds['train'])} training samples, {len(ds['test'])} test samples")

## 2. Format Each Example
def format_example(example):
    text = example['content'].strip()
    inputs = tokenizer(text, padding="max_length", max_length=256)
    inputs["labels"] = example["label"]
    return inputs

ds["train"] = ds["train"].map(format_example)
ds["test"] = ds["test"].map(format_example)
print("Dataset formatted.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset split: 1600 training samples, 400 test samples


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset formatted.


In [3]:
from transformers import AutoModelForSequenceClassification, AutoConfig
from peft import PeftModel

## 3. Load the base model WITH classification head
base_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
config = AutoConfig.from_pretrained(base_model_name)

# (e.g., binary classification → num_labels=2)
config.num_labels = 2
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name, config=config)

## 4. Load the Turnsense adapter on top of the base model
adapter_model_name = "latishab/turnsense"
model = PeftModel.from_pretrained(base_model, adapter_model_name)

print("Successfully loaded Turnsense with its adapter for classification!")

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-135M-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_config.json:   0%|          | 0.00/839 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/156M [00:00<?, ?B/s]

Successfully loaded Turnsense with its adapter for classification!


In [4]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorWithPadding

## 5. Setup SFTConfig
sft_config = SFTConfig(
    output_dir="outputs",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.0,
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=5,
    report_to=["wandb"],
    metric_for_best_model="loss",
    load_best_model_at_end=True,
    fp16=True,
    lr_scheduler_type="constant_with_warmup",
    optim="adamw_8bit",
    seed=3407
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
import wandb
from transformers import AutoTokenizer
wandb.finish()
wandb.init(project="huggingface", reinit=True)
import os
os.environ["WANDB_RUN_ID"] = wandb.util.generate_id()

## 6. Train the Model
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    data_collator=data_collator
)
trainer.train()

## 7. Save the fine-tuned adapter
adapter_save_path = "turnsense_finetuned_adapter"
model.save_pretrained(adapter_save_path)
print(f"Adapter saved to: {adapter_save_path}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: latishabesariani (latishabesariani-city-university-of-hong-kong) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

Truncating train dataset:   0%|          | 0/1600 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
50,0.089200,0.100178
100,0.087000,0.098571
150,0.084900,0.097203


TrainOutput(global_step=150, training_loss=0.08703718821207683, metrics={'train_runtime': 44.734, 'train_samples_per_second': 107.301, 'train_steps_per_second': 3.353, 'total_flos': 1071132180480000.0, 'train_loss': 0.08703718821207683})

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

## 8. Load tokenizer, base model, and fine-tuned adapter
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M-Instruct")
base_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name)
adapter_model_name = "turnsense_finetuned_adapter"
model = PeftModel.from_pretrained(base_model, adapter_model_name)
model.eval()

def predict_single_text(text, threshold=0.5):
    """
    Predict the label for a single piece of text based on the fine-tuned model.
    """
    # Tokenize the input text
    inputs = tokenizer(text, padding="max_length", max_length=256, return_tensors="pt")

    # Get model prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Apply softmax to get probabilities
    probs = torch.nn.functional.softmax(logits, dim=1)

    # Use the threshold for classification
    pred = 1 if probs[0][1].item() >= threshold else 0
    confidence = probs[0][pred].item()

    return pred, confidence

## 9. Test the fine-tuned model
while True:
    text_input = input("Enter a text to classify (or type 'exit' to stop): ")
    if text_input.lower() == 'exit':
        break
    try:
        prediction, confidence = predict_single_text(text_input)
        print(f"Prediction: {prediction} (Confidence: {confidence:.4f})")
    except Exception as e:
        print(f"Error during prediction: {e}")

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-135M-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Enter a text to classify (or type 'exit' to stop): hello
Prediction: 0 (Confidence: 0.6973)
Enter a text to classify (or type 'exit' to stop): so uhmmm...
Prediction: 0 (Confidence: 0.5762)
Enter a text to classify (or type 'exit' to stop): Hello, how are you?
Prediction: 1 (Confidence: 0.9925)


KeyboardInterrupt: Interrupted by user